In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse
from gensim import corpora, models, matutils
from  itertools import chain
from sklearn.feature_extraction import text 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#with open('/content/drive/My Drive/bgg_df.pickle', 'rb') as f:
#    df = pickle.load(f)

In [2]:
with open('bgg_df.pickle', 'rb') as f:
    df = pickle.load(f)

In [3]:
df['newcomments'].head()

KeyError: 'newcomments'

In [4]:
for idx, words in enumerate(df['comments']):
    combined = ""
    for item in words:
        combined += item
    df.at[idx,'newcomments'] = combined

In [5]:
my_additional_stop_words = ['game','games','play','players','played','player','plays','playing','just','great','like','good', 've','que','en','para','se','ks','le','el','es','really','la','com','don','http','https','www','com', \
                            '01','02','03','04','05','06','07','08','09','100','13','14','16','17','18','19','2015','2016','10','11','12','15','20','2017','2018','2019','2020','30']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

In [6]:
cv = CountVectorizer(stop_words=stop_words, strip_accents='ascii', max_df=.7,min_df=.04)
data_cv = cv.fit_transform(df.newcomments)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.index
data_dtm

,ability,able,absolutely,abstract,action,actions,actual,actually,add,added,...,work,works,world,worth,wouldn,written,wrong,year,years,yes
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
100086,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
corpus = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtm.transpose()))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())
word2id = dict((k, v) for k, v in cv.vocabulary_.items())
d = corpora.Dictionary()
d.id2token = id2word
d.token2id = word2id
lda = models.ldamulticore.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=11, passes=100,random_state=88)
lda.print_topics()

[(0,
  '0.057*"board" + 0.041*"abstract" + 0.041*"pieces" + 0.040*"tiles" + 0.021*"interesting" + 0.021*"strategy" + 0.016*"rules" + 0.014*"nice" + 0.014*"opponent" + 0.013*"simple"'),
 (1,
  '0.069*"fun" + 0.040*"simple" + 0.040*"little" + 0.037*"quick" + 0.033*"light" + 0.031*"filler" + 0.029*"luck" + 0.029*"nice" + 0.029*"easy" + 0.023*"fast"'),
 (2,
  '0.301*"dice" + 0.114*"roll" + 0.066*"die" + 0.053*"rolling" + 0.046*"luck" + 0.042*"pnp" + 0.033*"print" + 0.014*"solo" + 0.013*"fun" + 0.012*"free"'),
 (3,
  '0.088*"fun" + 0.032*"people" + 0.028*"party" + 0.027*"time" + 0.022*"group" + 0.015*"right" + 0.014*"best" + 0.013*"lot" + 0.012*"think" + 0.012*"better"'),
 (4,
  '0.022*"theme" + 0.022*"rules" + 0.019*"time" + 0.017*"fun" + 0.014*"long" + 0.014*"bit" + 0.012*"interesting" + 0.012*"think" + 0.012*"lot" + 0.011*"make"'),
 (5,
  '0.036*"kickstarter" + 0.027*"expansion" + 0.026*"love" + 0.026*"solo" + 0.019*"art" + 0.018*"fun" + 0.015*"new" + 0.015*"easy" + 0.015*"theme" + 0.014

In [8]:
dominant_topics_list = []

for i, row in enumerate(lda[corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0:
            dominant_topics_list.append(topic_num)

In [21]:
dominant_topics_list2 = []
for i, row in enumerate(lda[corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0:
            first_topic = topic_num
            first_prop = prop_topic
        elif j ==1:
            second_prop = prop_topic
    if first_prop == second_prop:
        dominant_topics_list2.append(11)
    else:
        dominant_topics_list2.append(first_topic)

In [16]:
topics_df = pd.DataFrame(dominant_topics_list,columns=['user_type'])

In [22]:
topics_df2 = pd.DataFrame(dominant_topics_list2,columns=['user_type'])

In [19]:
topics_df.head()

,user_type
0,0
1,9
2,5
3,0
4,0


In [23]:
topics_df2.head()

,user_type
0,11
1,9
2,5
3,11
4,11


In [27]:
for i in range(12):
    print(len(topics_df[topics_df['user_type']==i]))

44212
5235
2295
6362
10972
4572
7499
2607
3623
4731
7980
0


In [28]:
for i in range(12):
    print(len(topics_df2[topics_df2['user_type']==i]))

4057
5214
2297
6347
10966
4584
7488
2601
3620
4741
8017
40156


In [25]:
topics_df2[topics_df2['user_type']==11]

,user_type
0,11
3,11
4,11
21,11
50,11
...,...
100075,11
100080,11
100081,11
100082,11


In [14]:
topics_df.shape

(100088, 1)

In [35]:
rec_df = pd.concat([rec_df,topics_df2],axis=1)

In [38]:
rec_df.head()

,name,url,yr_pub,min_players,max_players,avg_play_time,min_play_time,max_play_time,min_age,complexity,...,Webcomics: Dork Tower,Webcomics: Penny Arcade,Word Games: First Letter Given,Word Games: Guess the Word,Word Games: Sign Language,Word Games: Spelling / Letters,avg_user_rating,has_expansion,avg_sent_score,user_type
0,Troia,https://boardgamegeek.com/boardgame/102580,2005.0,2,2,60,60,60,8,0.00,...,0,0,0,0,0,0,5.50000,0,0.000000,11
1,Mauseschlau & Bärenstark: Wissen & Lachen – Un...,https://boardgamegeek.com/boardgame/102595,2009.0,2,4,25,25,25,6,0.00,...,0,0,0,0,0,0,5.42857,0,-0.025600,9
2,Quintessential: The Fifth Element,https://boardgamegeek.com/boardgame/102597,2011.0,2,5,45,45,45,13,2.75,...,0,0,0,0,0,0,7.54167,0,0.482075,5
3,Wort für Wort,https://boardgamegeek.com/boardgame/102598,1998.0,1,5,25,25,25,6,0.00,...,0,0,0,0,0,0,5.08333,0,0.493900,11
4,Hannah Montana Secret Star,https://boardgamegeek.com/boardgame/102599,2009.0,2,4,15,15,15,6,0.00,...,0,0,0,0,0,0,4.00000,0,0.000000,11


In [33]:
rec_df = rec_df.drop('user_type',axis=1)

In [24]:
##with open('user_type_df2.pickle', 'wb') as user_type_df_pickle:
##    pickle.dump(topics_df2, user_type_df_pickle)

In [31]:
##with open('rec_df.pickle', 'rb') as f:
##    rec_df = pickle.load(f)

In [37]:
##with open('rec_df.pickle', 'wb') as rec_df_pickle:
##    pickle.dump(rec_df, rec_df_pickle)